Without NEL for now, just pattern matching, and linguistic heuristics to remove unpromising sentences from the pipeline



[ 1 ] Sentence Level check Filter before NER/NEL/neg Basic Tagger + Tokenizer
    => Len, SDP-len, Dep_complexity. TRIGGER WORDS! (cause, inhibit, therefore)
 
[ 2 ] Remaining sentences (with doc context needed for NER context??) =>


In [18]:
import spacy
from negspacy.negation import Negex #https://github.com/jenojp/negspacy
from spacy.pipeline import EntityRuler
from extractacy.extract import ValueExtractor

In [11]:
from scify.nlp import *

In [19]:
nlp = spacy.load("en_core_sci_md")
negex = Negex(nlp, ent_types=["PERSON","ORG"])
nlp.add_pipe(negex, last=True) #works on an per entity basis. If causal phrases become entities?


In [21]:
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_sm")
doc = nlp("This is a really nice, green apple. One apple a day ...!")
matcher = Matcher(nlp.vocab)

pattern = [{'ORTH': 'is'}]
for i in range(0,5): #aribtrary number of wild cards in between
    pattern.append({"OP": "?"}) 
pattern.append({"ORTH": "apple"})

matcher.add('test', None, pattern)
spans = [doc[start:end] for match_id, start, end in matcher(doc)]
for span in spans:
    print(spans)


[is a really nice, green apple]


In [ ]:
Doc.set_extension('my_ents', default=None)

#add to pipeline (for multiple docs...merging)
def move_ents_to_attr(doc):
    if doc._.my_ents is None:
        doc._.my_ents = []
    doc._.my_ents.extend(doc.ents)
    doc.ents = []
    return doc


In [20]:
#negex.get_patterns()

{'pseudo_patterns': [no further,
  not able to be,
  not certain if,
  not certain whether,
  not necessarily,
  without any further,
  without difficulty,
  without further,
  might not,
  not only,
  no increase,
  no significant change,
  no change,
  no definite change,
  not extend,
  not cause,
  not certain if,
  not certain whether,
  gram negative,
  not rule out,
  not ruled out,
  not been ruled out,
  not drain,
  no suspicious change,
  no interval change,
  no significant interval change],
 'preceding_patterns': [absence of,
  declined,
  denied,
  denies,
  denying,
  no sign of,
  no signs of,
  not,
  not demonstrate,
  symptoms atypical,
  doubt,
  negative for,
  no,
  versus,
  without,
  doesn't,
  doesnt,
  don't,
  dont,
  didn't,
  didnt,
  wasn't,
  wasnt,
  weren't,
  werent,
  isn't,
  isnt,
  aren't,
  arent,
  cannot,
  can't,
  cant,
  couldn't,
  couldnt,
  never,
  patient was not,
  without indication of,
  without sign of,
  without signs of,
  without

In [13]:
from operator import eq, ge, le

In [ ]:
PARAMS = {
    
    LEN_TRESH: 15,
    SUBTREE_LEN_TRESH: 6,
    
}

In [14]:
doc = nlp("Your mama is fat and you are fat too")

In [ ]:
texts = [
    
]

In [16]:
[len(sent) > LEN_TRESH for sent in doc.sents]

[9]